In [1]:
import sys
import os
import warnings

sys.path.append("../")
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

2023-05-02 21:35:14.305733: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
mass_train = pd.read_csv("../CBIS-DDSM/csv/mass_train_updated.csv", index_col=0).drop(columns=["index"], axis=1)
mass_test = pd.read_csv("../CBIS-DDSM/csv/mass_test_updated.csv", index_col=0).drop(columns=["index"], axis=1)
calc_train = pd.read_csv("../CBIS-DDSM/csv/calc_train_updated.csv", index_col=0).drop(columns=["index"], axis=1)
calc_test = pd.read_csv("../CBIS-DDSM/csv/calc_test_updated.csv", index_col=0).drop(columns=["index"], axis=1)

#### Creating new directory

In [4]:
mass_dataset_path = "../image_dataset/mass_dataset"
calc_dataset_path = "../image_dataset/calc_dataset"

if not os.path.exists(mass_dataset_path):
    os.makedirs(mass_dataset_path)

if not os.path.exists(calc_dataset_path):
    os.makedirs(calc_dataset_path)

folders_path = ["full_images", "cropped_images", "roi_images"]

for path in folders_path:
    full_path = os.path.join(mass_dataset_path, path)
    if not os.path.exists(full_path):
        os.makedirs(full_path)
        os.makedirs(os.path.join(full_path, "train"))
        os.makedirs(os.path.join(full_path, "train", "MALIGNANT"))
        os.makedirs(os.path.join(full_path, "train", "BENIGN"))
        os.makedirs(os.path.join(full_path, "validation"))
        os.makedirs(os.path.join(full_path, "validation", "MALIGNANT"))
        os.makedirs(os.path.join(full_path, "validation", "BENIGN"))

for path in folders_path:
    full_path = os.path.join(calc_dataset_path, path)
    if not os.path.exists(full_path):
        os.makedirs(full_path)
        os.makedirs(os.path.join(full_path, "train"))
        os.makedirs(os.path.join(full_path, "train", "MALIGNANT"))
        os.makedirs(os.path.join(full_path, "train", "BENIGN"))
        os.makedirs(os.path.join(full_path, "validation"))
        os.makedirs(os.path.join(full_path, "validation", "MALIGNANT"))
        os.makedirs(os.path.join(full_path, "validation", "BENIGN"))

#### Saving full images 

In [5]:
 def save_image_in_dir(dataset_info, save_path, train_test="train"):
        clahe=cv2.createCLAHE(clipLimit=40)
        print(len(dataset_info))
        i = 0
        for data in dataset_info:
            p_id, side, view, ab_id, image_path, label = data
            full_id = p_id + "_" + side + "_" + view + "_" + str(ab_id)
            image = cv2.imread(image_path, 0)
            gray_img_eqhist=cv2.equalizeHist(image)
            gray_clahe_image = cv2.cvtColor(clahe.apply(gray_img_eqhist), cv2.COLOR_GRAY2RGB)
            image = tf.image.resize(gray_clahe_image, (229, 229)).numpy().astype("uint8")
            save_dir = os.path.join(save_path, train_test, label, str(i)+'.jpg')
            cv2.imwrite(save_dir, image)
            i += 1

## Mass Cancer images saves

#### Full images

In [6]:
x, x_test, y, y_test = train_test_split(mass_train[["patient_id", 
                                                    "left or right breast",
                                                    "image view", 
                                                    "abnormality id", 
                                                    "image file path",
                                                    "pathology"]].values, mass_train["pathology"], test_size=0.2)

##### train

In [7]:
mass_train_path = os.path.join(mass_dataset_path, folders_path[0])
print(mass_train_path)
df = list(mass_train[["patient_id", 
                    "left or right breast",
                    "image view", 
                    "abnormality id", 
                    "image file path",
                    "pathology"]].values)

save_image_in_dir(dataset_info=df, 
                  save_path=mass_train_path,
                 train_test="train")
print("completed saving")

../image_dataset/mass_dataset/full_images
1214
Metal device set to: Apple M1


2023-05-02 21:35:31.746204: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-02 21:35:31.746908: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


completed saving


In [8]:
len(os.listdir("../image_dataset/mass_dataset/full_images/train/MALIGNANT/")) + \
len(os.listdir("../image_dataset/mass_dataset/full_images/train/BENIGN/"))

1713

##### Validation

In [9]:
mass_train_path = os.path.join(mass_dataset_path, folders_path[0])
print(mass_train_path)
save_image_in_dir(dataset_info=x_test, 
                  save_path=mass_train_path,
                 train_test="validation")
print("completed saving")

../image_dataset/mass_dataset/full_images
243
completed saving


In [10]:
len(os.listdir("../image_dataset/mass_dataset/full_images/validation/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/mass_dataset/full_images/validation/BENIGN/"))

382

#### Cropped Images

In [11]:
x, x_test, y, y_test = train_test_split(mass_train[["patient_id", 
                                                    "left or right breast",
                                                    "image view", 
                                                    "abnormality id", 
                                                    "cropped image file path",
                                                    "pathology"]].values, mass_train["pathology"], test_size=0.2)

##### Train

In [12]:
mass_train_path = os.path.join(mass_dataset_path, folders_path[1])
print(mass_train_path)
df = list(mass_train[["patient_id", 
            "left or right breast",
            "image view", 
            "abnormality id", 
            "cropped image file path",
            "pathology"]].values)
save_image_in_dir(dataset_info=df, 
                  save_path=mass_train_path,
                 train_test="train")
print("completed saving")

../image_dataset/mass_dataset/cropped_images
1214
completed saving


In [13]:
len(os.listdir("../image_dataset/mass_dataset/cropped_images/train/MALIGNANT/")) + \
len(os.listdir("../image_dataset/mass_dataset/cropped_images/train/BENIGN/"))

1702

##### Validation 

In [14]:
mass_train_path = os.path.join(mass_dataset_path, folders_path[1])
print(mass_train_path)
save_image_in_dir(dataset_info=x_test, 
                  save_path=mass_train_path,
                 train_test="validation")
print("completed saving")

../image_dataset/mass_dataset/cropped_images
243
completed saving


In [15]:
len(os.listdir("../image_dataset/mass_dataset/full_images/validation/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/mass_dataset/full_images/validation/BENIGN/"))

382

#### ROI images

In [16]:
x, x_test, y, y_test = train_test_split(mass_train[["patient_id", 
                                                    "left or right breast",
                                                    "image view", 
                                                    "abnormality id", 
                                                    "ROI mask file path",
                                                    "pathology"]].values, mass_train["pathology"], test_size=0.2)

##### Train

In [17]:
mass_train_path = os.path.join(mass_dataset_path, folders_path[2])
print(mass_train_path)
df = list(mass_train[["patient_id", 
                        "left or right breast",
                        "image view", 
                        "abnormality id", 
                        "ROI mask file path",
                        "pathology"]].values)
save_image_in_dir(dataset_info=df, 
                  save_path=mass_train_path,
                 train_test="train")
print("completed saving")

../image_dataset/mass_dataset/roi_images
1214
completed saving


In [18]:
len(os.listdir("../image_dataset/mass_dataset/roi_images/train/MALIGNANT/")) + \
len(os.listdir("../image_dataset/mass_dataset/roi_images/train/BENIGN/"))

1695

##### Validation

In [19]:
mass_train_path = os.path.join(mass_dataset_path, folders_path[2])
print(mass_train_path)
save_image_in_dir(dataset_info=x_test, 
                  save_path=mass_train_path,
                 train_test="validation")
print("completed saving")

../image_dataset/mass_dataset/roi_images
243
completed saving


In [20]:
len(os.listdir("../image_dataset/mass_dataset/roi_images/validation/MALIGNANT/")) + \
len(os.listdir("../image_dataset/mass_dataset/roi_images/validation/BENIGN/"))

364

## Calcification Images

### Full images

In [21]:
x, x_test, y, y_test = train_test_split(calc_train[["patient_id", 
                                                    "left or right breast",
                                                    "image view", 
                                                    "abnormality id", 
                                                    "image file path",
                                                    "pathology"]].values, calc_train["pathology"], 
                                        stratify=calc_train["pathology"], test_size=0.2)

##### Train

In [22]:
calc_train_path = os.path.join(calc_dataset_path, folders_path[0])
print(calc_train_path)
df = list(calc_train[["patient_id", 
                "left or right breast",
                "image view", 
                "abnormality id", 
                "image file path",
                "pathology"]].values)

save_image_in_dir(dataset_info=df, 
                  save_path=calc_train_path,
                 train_test="train")
print("completed saving")

../image_dataset/calc_dataset/full_images
1072
completed saving


In [23]:
len(os.listdir("../image_dataset/calc_dataset/full_images/train/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/calc_dataset/full_images/train/BENIGN/"))

1519

##### Validation

In [24]:
calc_train_path = os.path.join(calc_dataset_path, folders_path[0])
print(calc_train_path)
save_image_in_dir(dataset_info=x_test, 
                  save_path=calc_train_path,
                 train_test="validation")
print("completed saving")

../image_dataset/calc_dataset/full_images
215
completed saving


In [25]:
len(os.listdir("../image_dataset/calc_dataset/full_images/validation/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/calc_dataset/full_images/validation/BENIGN/"))

317

### cropped images

In [26]:
x, x_test, y, y_test = train_test_split(calc_train[["patient_id", 
                                                    "left or right breast",
                                                    "image view", 
                                                    "abnormality id", 
                                                    "cropped image file path",
                                                    "pathology"]].values, calc_train["pathology"], 
                                        stratify=calc_train["pathology"], test_size=0.2)

##### Train

In [27]:
calc_train_path = os.path.join(calc_dataset_path, folders_path[1])
print(calc_train_path)
df = list(calc_train[["patient_id", 
                "left or right breast",
                "image view", 
                "abnormality id", 
                "cropped image file path",
                "pathology"]].values)
save_image_in_dir(dataset_info=df, 
                  save_path=calc_train_path,
                 train_test="train")
print("completed saving")

../image_dataset/calc_dataset/cropped_images
1072
completed saving


In [28]:
len(os.listdir("../image_dataset/calc_dataset/cropped_images/train/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/calc_dataset/cropped_images/train/BENIGN/"))

1497

##### Validation

In [29]:
calc_train_path = os.path.join(calc_dataset_path, folders_path[1])
print(calc_train_path)
save_image_in_dir(dataset_info=x_test, 
                  save_path=calc_train_path,
                 train_test="validation")
print("completed saving")

../image_dataset/calc_dataset/cropped_images
215
completed saving


In [30]:
len(os.listdir("../image_dataset/calc_dataset/full_images/validation/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/calc_dataset/full_images/validation/BENIGN/"))

317

## ROI Images

In [31]:
x, x_test, y, y_test = train_test_split(calc_train[["patient_id", 
                                                    "left or right breast",
                                                    "image view", 
                                                    "abnormality id", 
                                                    "ROI mask file path",
                                                    "pathology"]].values, calc_train["pathology"], 
                                        stratify=calc_train["pathology"], test_size=0.2)

##### Train

In [32]:
calc_train_path = os.path.join(calc_dataset_path, folders_path[2])
print(calc_train_path)
df = list(calc_train[["patient_id", 
                    "left or right breast",
                    "image view", 
                    "abnormality id", 
                    "ROI mask file path",
                    "pathology"]].values)
save_image_in_dir(dataset_info=df, 
                  save_path=calc_train_path,
                 train_test="train")
print("completed saving")

../image_dataset/calc_dataset/roi_images
1072
completed saving


In [33]:
len(os.listdir("../image_dataset/calc_dataset/roi_images/train/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/calc_dataset/roi_images/train/BENIGN/"))

1519

##### Validation

In [34]:
calc_train_path = os.path.join(calc_dataset_path, folders_path[2])
print(calc_train_path)
save_image_in_dir(dataset_info=x_test, 
                  save_path=calc_train_path,
                 train_test="validation")
print("completed saving")

../image_dataset/calc_dataset/roi_images
215
completed saving


In [35]:
len(os.listdir("../image_dataset/calc_dataset/roi_images/validation/MALIGNANT/")) \
+ len(os.listdir("../image_dataset/calc_dataset/roi_images/validation/BENIGN/"))

323

## Test set generation

In [50]:
def generate_test_data(test_df, image_type_path):
    clahe=cv2.createCLAHE(clipLimit=40)
    labels = []
    images = []
    for image_path, label in mass_test[[image_type_path, "pathology"]].values:
        if label=="MALIGNANT":
            labels.append(1)
        if label=="BENIGN":
            labels.append(0)
        image = cv2.imread(image_path, 0)
        gray_img_eqhist=cv2.equalizeHist(image)
        gray_clahe_image = cv2.cvtColor(clahe.apply(gray_img_eqhist), cv2.COLOR_GRAY2RGB)
        image = tf.image.resize(gray_clahe_image, (229, 229)).numpy().astype("uint8")
        images.append(image)
    return images, labels

In [51]:
images, labels = generate_test_data(mass_test, image_type_path="image file path")

In [53]:
mass_test

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00016,4,LEFT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.245...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.308...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.308...
1,P_00016,4,LEFT,MLO,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,5,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.859...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.381...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.381...
2,P_00017,2,LEFT,CC,1,mass,ROUND,CIRCUMSCRIBED,4,MALIGNANT,4,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.221...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.212...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.212...
3,P_00017,2,LEFT,MLO,1,mass,ROUND,ILL_DEFINED,4,MALIGNANT,4,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.239...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.154...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.154...
4,P_00032,3,RIGHT,CC,1,mass,ROUND,OBSCURED,0,BENIGN,2,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.215...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.199...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.199...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,P_01815,3,RIGHT,MLO,1,mass,IRREGULAR,ILL_DEFINED,4,MALIGNANT,2,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.387...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.348...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.348...
337,P_01833,2,RIGHT,MLO,1,mass,IRREGULAR,ILL_DEFINED,5,MALIGNANT,5,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.355...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.183...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.183...
338,P_01865,2,LEFT,MLO,1,mass,IRREGULAR,ILL_DEFINED,4,MALIGNANT,2,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.413...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.358...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.358...
339,P_01912,3,RIGHT,CC,1,mass,IRREGULAR,SPICULATED,5,MALIGNANT,4,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.395...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.299...,../CBIS-DDSM/jpeg/1.3.6.1.4.1.9590.100.1.2.299...
